<a href="https://colab.research.google.com/github/jon-nowacki/Optimization-Models/blob/main/Transportation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Transporation Model Videos 13-14</h1>

***Pyomo Transportation Problem - Sets Parameters and Data File (Part 1)***

* https://www.youtube.com/watch?v=H8c8B9HvjQ0

***PYOMO Transportation Problem - Decision Variables, Objective Function,, and Constraints (Part 2)***
* https://www.youtube.com/watch?v=b8YfCIZY2xI


<h1> Step 1 - Install Pyomo</h1>
Start by installing the pyomo package.  You will need to do this at the beginning of every session.

In [ ]:
!pip install -q pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.1 MB/s eta 0:00:00


<h1>Step 2 - Install GLPK</h1>
Next, install GLPK. You will need to do this at the beginning of every session as well.

In [ ]:
!apt-get install -y -qq glpk-utils > /dev/null 2>&1

<h1> Step 3 - Create your model</h1>
Create your model. Here we are providing the start of the OJ Juice Model:




In [ ]:
# Lab 5 Pre-Lab: OJ Juice Company Abstract File
#Import Required Libraries
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

#create an abstract model object
model= AbstractModel()

# SETS
model.SUPPLIES = Set() # power supply at each node
model.DEMANDS = Set() # power demand at each node
#Parameters
model.s = Param(model.SUPPLIES) # power supply available at each plant
model.d = Param(model.DEMANDS) # the power demand at each factory
model.c = Param(model.SUPPLIES, model.DEMANDS) # the cost of transporting power

# Create Objective Function & Constraints

<img src="https://drive.google.com/uc?export=view&id=" width="700"/>

<img src="https://drive.google.com/uc?export=view&id=" width="700"/>

***PYOMO Transportation Problem - Decision Variables, Objective Function,, and Constraints (Part 2)***
* https://www.youtube.com/watch?v=b8YfCIZY2xI


In [ ]:
# Decision variables
model.x = Var(model.SUPPLIES,model.DEMANDS,within=NonNegativeReals)

# Objective Function
def objective_rule(model):
  return sum(model.c[i,j]*model.x[i,j] for i in model.SUPPLIES for j in model.DEMANDS)
model.minCost = Objective(rule=objective_rule,sense=minimize)

In [ ]:
# Subject to / Constraint
def supply_rule(model,i):
  return sum(model.x[i,j] for j in model.DEMANDS)==model.s[i]
model.supplyConstraints = Constraint(model.SUPPLIES, rule=supply_rule)

In [ ]:
# Subject to / Constraint
def demand_rule(model,j):
  return sum(model.x[i,j] for i in model.SUPPLIES)==model.d[j]
model.demandConstraints = Constraint(model.DEMANDS, rule=demand_rule)

# Import the dat file

***Pyomo Transportation Problem - Sets Parameters and Data File (Part 1)***

https://www.youtube.com/watch?v=H8c8B9HvjQ0&t=10s

Dat file: transportation.dat

```
param: SUPPLIES: s:=
PLANT1 35
PLANT2 50
PLANT3 40;

param: DEMAND: d:=
City1 45
City2 20
City3 30
City4 30;

param: c: City1 City2 City3 City4:=
PLANT1 8 6 10 9
PLANT2 9 12 13 7
PLANT3 14 9 16 5;
```
Or define more than one parameter at the same time


```

```
For a more condensed dat file

In [ ]:

data_file='/content/transportation.dat'
data = DataPortal()
# ???? why model=model???
data.load(filename=data_file, model=model)
instance=model.create_instance(data)
#instance.pprint()

In [ ]:
import pandas as pd
df=pd.read_csv(data_file)
df.head()

,set QUALITIES := 6 9;
0,set PRODUCTS := juice bags;
1,param: profit requiredQuality:=
2,juice 0.45 8
3,bags 0.30 7;
4,param available:=


In [ ]:
instance.pprint()

4 Set Declarations
    DEMANDS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'City1', 'City2', 'City3', 'City4'}
    SUPPLIES : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'PLANT1', 'PLANT2', 'PLANT3'}
    c_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain           : Size : Members
        None :     2 : SUPPLIES*DEMANDS :   12 : {('PLANT1', 'City1'), ('PLANT1', 'City2'), ('PLANT1', 'City3'), ('PLANT1', 'City4'), ('PLANT2', 'City1'), ('PLANT2', 'City2'), ('PLANT2', 'City3'), ('PLANT2', 'City4'), ('PLANT3', 'City1'), ('PLANT3', 'City2'), ('PLANT3', 'City3'), ('PLANT3', 'City4')}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain           : Size : Members
        None :     2 : SUPPLIES*DEMANDS :   12 : {('PLANT1', 'City1'), ('PLANT1', 'City2'), ('PLANT1', 'City3'), ('PLANT1', 'C

In [ ]:
optimizer=SolverFactory( 'glpk')
# optimizer=SolverFactory( 'glpk',executable='/usr/bin/glpsol')
results=optimizer.solve(instance)

In [ ]:
instance.display()

Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
        ('PLANT1', 'City1') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT1', 'City2') :     0 :  10.0 :  None : False : False : NonNegativeReals
        ('PLANT1', 'City3') :     0 :  25.0 :  None : False : False : NonNegativeReals
        ('PLANT1', 'City4') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City1') :     0 :  45.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City2') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City3') :     0 :   5.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City4') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT3', 'City1') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT3', 'City2') :     0 :  10.0 :  None : False : False : NonNegative

# Error Catching


In [ ]:
results=optimizer.solve(instance)                #tells your optimizer to solve the model object


if(results.solver.status==SolverStatus.ok) and (results.solver.termination_condition.optimal):
  #unbounded.display()                       #writes results to the console so you can see them
  print("\t Model Results")
  instance.display()
elif(results.solver.termination_condition==TerminationCondition.infeasible) or results.solver.termination_condition==TerminationCondition.other:
  print("MODEL INFEASIBLE. Check Constraints")

else:
  print("Solver status", results.solver.status)
  print("Termination Condition", results.solver.termination_condition)

	 Model Results
Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
        ('PLANT1', 'City1') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT1', 'City2') :     0 :  10.0 :  None : False : False : NonNegativeReals
        ('PLANT1', 'City3') :     0 :  25.0 :  None : False : False : NonNegativeReals
        ('PLANT1', 'City4') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City1') :     0 :  45.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City2') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City3') :     0 :   5.0 :  None : False : False : NonNegativeReals
        ('PLANT2', 'City4') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT3', 'City1') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('PLANT3', 'City2') :     0 :  10.0 :  None : False : Fal